In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("medicine.csv")

In [ ]:
data.sample(10)

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [6]:
data.dropna(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data['Description']

In [11]:
data['Description'] = data['Description'].apply(lambda x : x.split())
data['Reason'] = data['Reason'].apply(lambda x : x.split())

In [ ]:
data.tail()

In [13]:
data['Description'] = data['Description'].apply(lambda x :[i.replace(" ", "") for i in x] )

In [14]:
data['tags'] = data['Description'] + data['Reason']

In [ ]:
data.head()

In [16]:
new_data = data[['index', 'Drug_Name', 'tags']]

In [ ]:
new_data.head()

In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x : " ".join(x))

In [ ]:
new_data['tags'] = new_data['tags'].apply(lambda x : x.lower())

In [ ]:
new_data.head()

In [21]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = "english", max_features = 5000)

In [23]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
stem("You are so so beautiful")

In [ ]:
new_data['tags'] = new_data['tags'].apply(stem)

In [ ]:
new_data.head()

In [27]:
vectors = cv.fit_transform(new_data['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
cv.get_feature_names_out()

In [30]:
from sklearn.metrics.pairwise import cosine_similarity


In [31]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [33]:
def recommendation(medicine):
    medicine_index = new_data[new_data['Drug_Name'] == medicine].index[0]
    distance = similarity[medicine_index]
    medicine_list = sorted(list(enumerate(distance)), reverse=True, key = lambda x : x[1])[1:6]
    for i in medicine_list:
        print(new_data.iloc[i[0]].Drug_Name)

In [ ]:
recommendation("Ocudor Eye Drops 5ml")

In [35]:
import pickle
pickle.dump(new_data.to_dict(), open("medicine_dict.pkl", "wb"))
pickle.dump(similarity, open("similarity.pkl", "wb"))